# Valkyrie Point Cloud Data Access



The Valkyrie project provides web services for ordering spatially and temporally subsetted Lidar point cloud data from the [BLATM L1B](https://nsidc.org/data/BLATM1B), [ILATM L1B v1](https://nsidc.org/data/ilatm1b/versions/1), [ILATM L1B V2](https://nsidc.org/data/ILATM1B), [ILVIS2](https://nsidc.org/data/ILVIS2) and [IceSat GLAH06](https://nsidc.org/data/GLAH06/) data products. The following table describes the temporal and spatial coverage of each of these dataset as well as the sensor and platform used to acquire the data.


---

|              | Spatial Coverage                                                      | Temporal Coverage                              | Platform                                              | Sensor                   |
|--------------|-----------------------------------------------------------------------|------------------------------------------------|-------------------------------------------------------|--------------------------|
| BLATM L1B    | South: N:-53, S: -90, E:180, W:-180 North: N:90, S: 60, E:180, W:-180 | 23 June 1993 - 30 October 2008                 | DC-8, DHC-6, P-3A ORION, P-3B                         | ATM                      |
| ILATM L1B V1 | South: N:-53, S: -90, E:180, W:-180 North: N:90, S: 60, E:180, W:-180 | 31 March 2009 - 8 November 2012 (updated 2013) | AIRCRAFT, DC-8, P-3B                                  | ATM                      |
| ILATM L1B V2 | South: N:-53, S: -90, E:180, W:-180 North: N:90, S: 60, E:180, W:-180 | 20 March 2013 - 16 May 2019 (updated 2020)     | C-130, DC-8, HU-25A, HU-25C, P-3B, WP-3D ORION        | ATM                      |
| ILVIS2       | North: N:90, S: 60, E:180, W:-180                                     | 25 August 2017 - 20 September 2017             | AIRCRAFT, B-200, C-130, DC-8, G-V, HU-25C, P-3B, RQ-4 | ALTIMETERS, LASERS, LVIS |
| GLAH06       | Global: N:86, S: -86, E:180, W:-180                                     |     20 February 2003 - 11 October 2009        | IceSat | ALTIMETERS, CD, GLAS, GPS, GPS Receiver, LA, PC


--- 

> Note: If you have any qustions about the data please contact NSIDC user services at users@nsidc.org

In this tutorial we are going to use iPyLeaflet and other Jupyter widgets to select our constraints and put a data request to our Valkyrie API.

First to get you familiarized with the combined coverage of these products run the next cell and play the Youtube video.




In [ ]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('jRB1OEDXXwY')

In [1]:
import ipywidgets as widgets
r = widgets.Dropdown(
    options=['south', 'north'],
    description='Hemisphere:',
    disabled=False,
)
# Now let's pick up an hemisphere
display(r)

Dropdown(description='Hemisphere:', options=('south', 'north'), value='south')

In [2]:
# The UI is encapsulated in this helper file, 
# if you see controls repeated try running the cell again: https://github.com/ipython-contrib/jupyter_contrib_nbextensions/issues/1056
import valkyrie_utils as vu
from ipyleaflet import (
    Map,
    Polygon,
    GeoJSON,
    DrawControl
)

dc = DrawControl(circlemarker={},
                 polyline={},
                 polygon={},
                 rectangle = {
                    "shapeOptions": {
                        "fillColor": "#fca45d",
                        "color": "#fca45d",
                        "fillOpacity": 0.5
                    }
})
h = vu.hemisphere[r.value]
base_map = Map(
        center=h['center'],
        zoom=1,
    max_zoom=18,
    min_zoom=1,
        basemap=h['base_map'],
        crs=h['projection'])
base_map.add_control(dc)
display(vu.dataset, vu.ITRF, vu.date_range_slider, base_map)

Dropdown(description='Dataset:', options=('ATM1B', 'GLAH06', 'ILVIS2'), value='ATM1B')

Dropdown(description='ITRF:', options=('', 'ITRF2000', 'ITRF2008', 'ITRF2014'), value='')

SelectionRangeSlider(description='Date Range', index=(0, 9860), layout=Layout(width='100%'), options=((' 1993-…

Map(center=[90, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

### Now that we have a dataset selected and we are contraining our search using a temporal range and polygon we can call the Valkyrie API

> Note: Valkyrie uses Hermes, NSIDC's data ordering system.

In [ ]:
# Some reference dates for ATM missions are Aug-Dec 2014 in the Antarctic peninsula.
# params = vu.build_params(dc)
# print(vu.dataset.value, params,)
dc.last_draw

In [3]:
import json
import os
import requests

def feature_info(event, **kwargs):
    print(kwargs.get('feature'))
    print('whats up')

if not os.path.exists('europe_110.geo.json'):
    url = 'https://github.com/jupyter-widgets/ipyleaflet/raw/master/examples/europe_110.geo.json'
    r = requests.get(url)
    with open('europe_110.geo.json', 'w') as f:
        f.write(r.content.decode("utf-8"))

with open('europe_110.geo.json', 'r') as f:
    data = json.load(f)
geo_json = GeoJSON(data=data, style = {'color': 'green', 'opacity':1, 'weight':1, 'dashArray':'9', 'fillOpacity':0.1})

base_map.add_layer(geo_json)

geo_json.on_click(feature_info)

## Sending a data order

Now that we have our constraints we just need to post our order and wait for Valkyrie to fulfill it

In [ ]:
import requests
base_url = f'http://staging.valkyrie-vm.apps.nsidc.org/1.0/{vu.dataset.value}'
response = requests.post(base_url, params=params)
# now we are going to print the response from Valkyrie
status_url = response.json()['status_url']
print(status_url, response)

In [ ]:
base_map

## Downloading the data
Let's get some coffee, some Valkyrie orders are in the Gigabytes real amd may take a little while to be processed. 
Once that your status URL says is completed we can grab the HDF5 data file using the URL on the same response!

In [ ]:
order_status = requests.get(status_url).json()
print(order_status)

In [19]:
from ipyleaflet import Map, GeoJSON, projections, WMSLayer, basemaps
import json
import os
import requests

if not os.path.exists('europe_110.geo.json'):
    url = 'https://github.com/jupyter-widgets/ipyleaflet/raw/master/examples/europe_110.geo.json'
    r = requests.get(url)
    with open('europe_110.geo.json', 'w') as f:
        f.write(r.content.decode("utf-8"))

with open('europe_110.geo.json', 'r') as f:
    data = json.load(f)

# m = Map(center=(50.6252978589571, 0.34580993652344), zoom=0, crs=projections.EPSG3413)
geo_json = GeoJSON(data=data, style = {'color': 'green', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1})



fixed_north = {
    'name': 'EPSG:3413',
    'custom': True, #This is important, it tells ipyleaflet that this projection is not on the predefined ones.
    'proj4def': '+proj=stere +lat_0=90 +lat_ts=70 +lon_0=-45 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs',
    'origin': [-4194304, 4194304],
    'bounds': [
        [-4194304, -4194304],
        [4194304, 4194304]   
    ],
    'resolutions': [
        8192.0 * 2,
        8192.0,
        4096.0,
        2048.0,
        1024.0,
        512.0,
        256.0
    ]
}


m2 = Map(center=(90, 0),
        zoom=1,
         max_zoom=5,
         min_zoom=1,
        basemap=basemaps.NASAGIBS.BlueMarble3413,
        crs=fixed_north)


dc2 = DrawControl(marker={'shapeOptions': {'color': '#0000FF'}})

m2.add_control(dc2)
m2.add_layer(geo_json)
m2

Map(center=[90, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…